In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import math

import torch
from allennlp.data.tokenizers import WordTokenizer

from tuple_filtering.tuple_dataset_reader import TupleDatasetReader

In [ ]:
torch.manual_seed(1)

In [13]:
reader = TupleDatasetReader(lazy=True)
tuple_dataset = reader.read('data/openbook_tuples.tsv')

In [ ]:
class TupleEmbedder()